# Running Marlowe Contracts without Blockchain Transactions

This lecture shows how to execute a contract using `marlowe-cli`, but without submitting transactions on the blockchain. This lets one simulate the operation of a contract.

## Escrow Contract

*   A buyer wishes to purchase an item.
*   They deposit funds.
*   After they deposit funds they may report a problem with the purchase.
*   If they don’t report a problem, then the funds are released to the seller.
*   If they do report a problem, the seller may agree that there is a problem (in which case the buyer receives a refund) or they may dispute that there is a problem.
*   If the seller disputes the problem, then a mediator decides who receives the funds.
*   The contract has logic to handle situations where a party fails to act in a timely manner.

### Flow Chart for Escrow Example

![Execution paths through the escrow example contract.](diagrams/escrow-flow.png)

### Escrow Example in Marlowe Format

```haskell
When
  [
    Case (Deposit (Role "Francis Beaumont (the seller)") (Role "Thomas Middleton (the buyer)") ada 256)
      ( When
        [
          Case (Choice (ChoiceId "Everything is alright" (Role "Thomas Middleton (the buyer)")) [Bound 0 0])
            Close
        , Case (Choice (ChoiceId "Report problem" (Role "Thomas Middleton (the buyer)")) [Bound 1 1])
            ( Pay (Role "Francis Beaumont (the seller)") (Account (Role "Thomas Middleton (the buyer)")) ada 256 )
            ( When
              [
                Case (Choice (ChoiceId "Confirm problem" (Role "Francis Beaumont (the seller)")) [Bound 1 1])
                  Close
              , Case (Choice (ChoiceId "Dispute problem" (Role "Francis Beaumont (the seller)")) [Bound 0 0])
                ( When
                  [
                    Case (Choice (ChoiceId "Dismiss claim" (Role "Christopher Marlowe (the mediator)")) [Bound 0 0])
                      ( Pay (Role "Thomas Middleton (the buyer)") (Account (Role "Francis Beaumont (the seller)")) ada 256 )
                      Close
                  , Case (Choice (ChoiceId "Confirm claim" (Role "Christopher Marlowe (the mediator)")) [Bound 1 1])
                      Close
                  ]
                  (SlotParam "Mediation deadline")
                  Close
                )
              ]
              (SlotParam "Dispute deadline")
            ) Close
        ]
        (SlotParam "Complaint deadline")
        Close
      )
  ]
  (SlotParam "Payment deadline")
  Close
```

### Escrow Example in Blockly

![Blockly representation of escrow example contract, split into three columns.](diagrams/escrow-blockly.png)

### Four of the Eight Pathways through the Escrow Contract

![Four possible executions of the escrow contract.](diagrams/escrow-paths.png)

😨 ***REDRAW AND REVISE LAYOUT***

## Scenario

![Overview of escrow example.](diagrams/escrow-detailed-setup.png)

😨 ***REDRAW AND REVISE LAYOUT***

## Select the Parameters for the Contract

First use some environment variables to store some values

In [1]:
INITIAL_LOVELACE=3000000             # The creation transaction will deposit 3₳.
PRICE=256000000                      # The prices of the item is 256₳.

SELLER_ROLE=FB                       # Francis Beaumont (FB) is the seller.
BUYER_ROLE=TM                        # Thomas Middleton is the buyer.
MEDIATOR_ROLE=CM                     # Christopher Marlowe is the mediator.

NOW=$(($(date -u +%s)*1000))         # The current time in POSIX milliseconds.
HOUR=$((60*60*1000))                 # One hour, in POSIX milliseconds.
PAYMENT_DEADLINE=$((NOW+10*HOUR))    # The payment deadline, ten hours from now.
COMPLAINT_DEADLINE=$((NOW+12*HOUR))  # The complaint deadline, twelve hours from now.
DISPUTE_DEADLINE=$((NOW+14*HOUR))    # The dispute deadline, fourteen hours from now.
MEDIATION_DEADLINE=$((NOW+16*HOUR))  # The mediation deadline, sixteen hours from now.

## Create the Contract and Its Initial State

Now create the contract:
*   The contract is stored in the JSON file `tx-1.contract`.
*   The initial state is stored in the JSON file `tx-1.state`.

In [2]:
marlowe-cli template escrow --minimum-ada "$INITIAL_LOVELACE"          \
                            --price "$PRICE"                           \
                            --seller "Role=$SELLER_ROLE"               \
                            --buyer "Role=$BUYER_ROLE"                 \
                            --mediator "Role=$MEDIATOR_ROLE"           \
                            --payment-deadline "$PAYMENT_DEADLINE"     \
                            --complaint-deadline "$COMPLAINT_DEADLINE" \
                            --dispute-deadline "$DISPUTE_DEADLINE"     \
                            --mediation-deadline "$MEDIATION_DEADLINE" \
                            --out-contract-file tx-1.contract          \
                            --out-state-file    tx-1.state

## Alternative: Download a Contract from Marlowe Playground

Instead of using the `marlowe-cli template` command, one can create a contract in Marlowe Playground using Haskell, JavaScript, or Blockly.
1.  Design the contract in Marlowe Playground.
2.  Press the "Send to Simulator" button.
3.  Click "Download as JSON" to download the contract, saving it as `tx-1.contract`.

One also needs to create the initial state `tx-1.state`, which consists of (a) account balances, (b) preexisting choices, (c) preexisting assignments of variables, and the minimum POSIX time when the contract can be started.

Simply create a JSON file with no choices or bound values and just one account with the minimum ADA deposit for the role that creates the contract (in this case, `$MEDIATOR_ROLE`).

In [3]:
cat << EOI > tx-1.state
{
    "accounts": [
        [[{"role_token": "$MEDIATOR_ROLE"}, {"currency_symbol": "", "token_name": ""}], $INITIAL_LOVELACE]
    ],
    "choices": [],
    "boundValues": [],
    "minTime": 1
}
EOI

## Minting the Role Currency

Previously we created a wallet with a signing key file `my-wallet.skey` and address file `my-wallet.address`. We also located the Cardano node socket path at `$CARDANO_NODE_SOCKET_PATH`. Now we use these to mint role tokens for use in the Marlowe contract.

In [4]:
export CARDANO_NODE_SOCKET_PATH=~/.local/share/Daedalus/marlowe_pioneers/cardano-node.socket

In [5]:
marlowe-cli util mint --testnet-magic 1567                      \
                      --socket-path "$CARDANO_NODE_SOCKET_PATH" \
                      --required-signer my-wallet.skey          \
                      --change-address $(cat my-wallet.address) \
                      --out-file /dev/null                      \
                      --submit 600                              \
                      "$MEDIATOR_ROLE" "$SELLER_ROLE" "$BUYER_ROLE"

PolicyID "38fa7eb06a9bc7f219c4a7e06aaa37f2d92569f6ef350c42323910fa"


The policy ID will be used for the Marlowe roles currency.

In [6]:
ROLES_CURRENCY=38fa7eb06a9bc7f219c4a7e06aaa37f2d92569f6ef350c42323910fa

We can query the address to see that the tokens have been minted. These will also be visible in Daedalus.

In [7]:
cardano-cli query utxo --testnet-magic 1567 --address $(cat my-wallet.address)

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
7f2728cdf78b21b9da70851a16f340a94d64302957eb76a09f8a821a7e2769a7     0        969821035 lovelace + TxOutDatumNone
7f2728cdf78b21b9da70851a16f340a94d64302957eb76a09f8a821a7e2769a7     1        10000000 lovelace + 1 38fa7eb06a9bc7f219c4a7e06aaa37f2d92569f6ef350c42323910fa.434d + TxOutDatumNone
7f2728cdf78b21b9da70851a16f340a94d64302957eb76a09f8a821a7e2769a7     2        10000000 lovelace + 1 38fa7eb06a9bc7f219c4a7e06aaa37f2d92569f6ef350c42323910fa.4642 + TxOutDatumNone
7f2728cdf78b21b9da70851a16f340a94d64302957eb76a09f8a821a7e2769a7     3        10000000 lovelace + 1 38fa7eb06a9bc7f219c4a7e06aaa37f2d92569f6ef350c42323910fa.544d + TxOutDatumNone


## Step 1: Mediator Creates Escrow Contract with Initial ADA

![Initialization for step 1.](diagrams/escrow-step1.png)

😨 ***REDRAW AND REVISE LAYOUT***

## Initializing Step 1

Marlowe CLI uses `.marlowe` JSON files to store information about the progression of a Marlowe contract. Now that we have created the contract, we can bundle the contract, state, Plutus data, and network information into such a file.

In [8]:
marlowe-cli run initialize --testnet-magic 1567                      \
                           --socket-path "$CARDANO_NODE_SOCKET_PATH" \
                           --roles-currency "$ROLES_CURRENCY"        \
                           --contract-file tx-1.contract             \
                           --state-file    tx-1.state                \
                           --out-file      tx-1.marlowe              \
                           --print-stats


Validator size: 12633
Base-validator cost: ExBudget {exBudgetCPU = ExCPU 24920101, exBudgetMemory = ExMemory 83800}


In [9]:
ls -l tx-1.*

-rw-rw-r-- 1 pioneer marlowe  8286 Apr 29 10:17 tx-1.contract
-rw-rw-r-- 1 pioneer marlowe 40778 Apr 29 10:17 tx-1.marlowe
-rw-rw-r-- 1 pioneer marlowe   348 Apr 29 10:17 tx-1.state


## Step 2: Buyer Deposits Funds into Seller’s Account

![Transition from step 1 to step 2.](diagrams/escrow-step2.png)

😨 ***REDRAW AND REVISE LAYOUT***

## Transition from Step 1 to Step 2

In [10]:
marlowe-cli run prepare --marlowe-file tx-1.marlowe           \
                        --deposit-account "Role=$SELLER_ROLE" \
                        --deposit-party "Role=$BUYER_ROLE"    \
                        --deposit-amount "$PRICE"             \
                        --invalid-before "$NOW"               \
                        --invalid-hereafter "$((NOW+9*HOUR))" \
                        --out-file tx-2.marlowe               \
                        --print-stats


Datum size: 463


## Step 3: The Buyer Reports That There is a Problem

![Transition from step 2 to step 3.](diagrams/escrow-step3.png)

😨 ***REDRAW AND REVISE LAYOUT***

## Transition from Step 2 to Step 3

In [11]:
marlowe-cli run prepare --marlowe-file tx-2.marlowe           \
                        --choice-name "Report problem"        \
                        --choice-party "Role=$BUYER_ROLE"     \
                        --choice-number 1                     \
                        --invalid-before "$NOW"               \
                        --invalid-hereafter "$((NOW+9*HOUR))" \
                        --out-file tx-3.marlowe               \
                        --print-stats


Datum size: 341
Payment 1
  Acccount: "FB"
  Payee: Account "TM"
  Ada: 256.000000


## Step 4: The Seller Disputes that There is a Problem

![Transition from step 3 to step 4.](diagrams/escrow-step4.png)

😨 ***REDRAW AND REVISE LAYOUT***

## Transition from Step 3 to Step 4

In [12]:
marlowe-cli run prepare --marlowe-file tx-3.marlowe            \
                        --choice-name "Dispute problem"        \
                        --choice-party "Role=$SELLER_ROLE"     \
                        --choice-number 0                      \
                        --invalid-before "$NOW"                \
                        --invalid-hereafter "$((NOW+9*HOUR))"  \
                        --out-file tx-4.marlowe                \
                        --print-stats


Datum size: 262


## Step 5: The Mediator Dismisses the Claim

![Transition from step 4 to step 5](diagrams/escrow-step5.png)

😨 ***REDRAW AND REVISE LAYOUT***

## Transition from Step 4 to Step 5

In [13]:
marlowe-cli run prepare --marlowe-file tx-4.marlowe           \
                        --choice-name "Dismiss claim"         \
                        --choice-party "Role=$MEDIATOR_ROLE"  \
                        --choice-number 0                     \
                        --invalid-before "$NOW"               \
                        --invalid-hereafter "$((NOW+9*HOUR))" \
                        --out-file tx-5.marlowe               \
                        --print-stats


Datum size: 104
Payment 1
  Acccount: "TM"
  Payee: Account "FB"
  Ada: 256.000000
Payment 2
  Acccount: "CM"
  Payee: Party "CM"
  Ada: 3.000000
Payment 3
  Acccount: "FB"
  Payee: Party "FB"
  Ada: 256.000000


## Troubleshooting

*   Run the contract in the Marlowe Playground simulator before running it at the command line.
*   Pay attention to the timeouts in `When` clauses of a contract:
    *   Does the timeout trigger the intendent movement of funds between accounts?
    *   Is the timeout consistent with other timeouts in `Case` statements?
*   At the semantic level, these Marlowe errors can occur:
    *   `TEAmbiguousTimeIntervalError`: The time interval for a transaction straddles the timeout of a `When` clause.
    *   `TEApplyNoMatchError`: The attempted application of input to the contract was illegal.
    *   `TEIntervalError IntervalError`
        *   `InvalidInterval`: The start of the time interval is after its end.
        *   `IntervalInPast`: The interval falls before the current time.
    *   `TEUselessTransaction`: The attempted application of input does not change the state of the contract.
    *   `TEHashMismatch`: An incorrect contract was provided for the merkleized continuation.

## Resources

*   Marlowe Debugging Cookbook: <<https://github.com/input-output-hk/marlowe-cardano/blob/mpp-cli-lectures/marlowe/debugging-cookbook.md>>.
*   Example contracts with `marlowe-cli`: <<https://github.com/input-output-hk/marlowe-cardano/blob/mpp-cli-lectures/marlowe-cli/examples/ReadMe.md>>.
*   Marlowe contract examples: <<https://github.com/input-output-hk/marlowe-cardano/tree/main/marlowe-contracts>>.
*   Marlowe Playground: <<https://playground.marlowe.iohkdev.io/#/>>.
*   Marlowe Run
    *   On pioneers testnet: <<https://marlowe-run-marlowe-pioneers.plutus.aws.iohkdev.io/>>.
    *   On mock network: <<https://marlowe-finance.io/>>.
*   Cardano Docs for Marlowe: <<https://docs.cardano.org/marlowe/learn-about-marlowe>>.
*   Plutus Pioneers Program: <<https://github.com/input-output-hk/plutus-pioneer-program>>.
*   Plutus Community: <<https://plutus-community.readthedocs.io/en/latest/>>.
*   The Plutonomicon: <<https://github.com/Plutonomicon/plutonomicon/blob/main/README.md>>.

## Summary

*   Use `marlowe-cli template` instantiate a contract from a template.
*   Alternatively, use Marlowe Playground to design a contract and download it as a JSON file.
*   The initial state for a contract consists of . . .
    *    Account balances.
    *    Preexisting choices.
    *    Preexisting assignments of variables.
    *    The minimum POSIX time when the contract can be started.
*   Use `marlowe-cli run initialize` to package the contract, initial state, Plutus data, and network information into a `.marlowe` file that can be used to run the contract.
*   Use `marlowe-cli run prepare` to apply input to a contract, which causes it to transition from one state to another.
*   Use `marlowe-cli util mint` if you need to mint role tokens.

## Other Lectures

Lectures on Marlowe CLI: <<https://github.com/input-output-hk/marlowe-cardano/blob/mpp-cli-lectures/marlowe-cli/lectures/ReadMe.md>>

*   Overview of Marlowe CLI
*   Installing Marlowe CLI and Associated Tools
*   ~~Running Marlowe Contracts without Blockchain Transactions~~
*   Running Marlowe Contacts on the Blockchain
*   Running Marlowe Contracts with the Plutus Application Backend (PAB)
*   Reference for Marlowe CLI Commands